In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup as bs

In [3]:
LIMIT = 50
# indeed url
Q = "spring"
INDEED_URL = f"https://kr.indeed.com/jobs?q={Q}&limit={LIMIT}"

In [4]:
# test indeed_page
def extract_indeed_pages():
  get_request = requests.get(INDEED_URL)
  html_parse = bs(get_request.text, 'html.parser') # HTML 파싱

  # pagination > a > span 찾기
  pages = html_parse.find("div", {"class": "pagination"}).find_all('a')

  page_list = []

  # 마지막 값(Next)을 제외한 page를 저장
  for page in pages[:-1]:
    page_value = int(page.find('span').string)
    page_list.append(page_value)

  max_page = page_list[-1]
  return max_page

In [5]:
# job
def extract_jobs(html):
  # title
  h2_title = html.find("h2", {"class": "title"})
  title = h2_title.find("a")["title"]
  # company
  span_company = html.find("span", {"class": "company"})
  a_company = span_company.find("a")
  
  company = ""
  if a_company is not None:
    company = str(a_company.string)
  else:
    company = str(span_company.string)
  company = company.strip()
  location = html.find("div", {"class": "recJobLoc"})["data-rc-loc"]
  job_id = html["data-jk"]
  return {"title": title, "company": company, "location": location, "link": f"https://kr.indeed.com/viewjob?jk={job_id}"}

In [6]:
# indeed
def extract_indeed_html(last_page):
  jobs = []
  for page in range(last_page):
    print(f" scrapping {page}")
    # https://kr.indeed.com/jobs?q=python&limit=50&start=0
    get_request = requests.get(f"{INDEED_URL}&start={page*LIMIT}")
    print(f" request status: {get_request.status_code}")

    html_parse = bs(get_request.text, 'html.parser') # HTML 파싱
    results = html_parse.find_all("div", {"class": "jobsearch-SerpJobCard"})
    for result in results:
      job = extract_jobs(result)
      jobs.append(job)
  return jobs

In [7]:
max_page = extract_indeed_pages()
indeed_jobs = extract_indeed_html(max_page)
indeed_jobs

 scrapping 0
 request status: 200
 scrapping 1
 request status: 200
 scrapping 2
 request status: 200
 scrapping 3
 request status: 200
 scrapping 4
 request status: 200
 scrapping 5
 request status: 200
 scrapping 6
 request status: 200
 scrapping 7
 request status: 200


[{'title': '주한 미국 대사관 상무부 인턴쉽 채용 안내',
  'company': '성균관대학교',
  'location': '서울',
  'link': 'https://kr.indeed.com/viewjob?jk=daa9f53d61644f2c'},
 {'title': '전산팀 신입/경력사원',
  'company': '한국유나이티드제약',
  'location': '서울 논현동',
  'link': 'https://kr.indeed.com/viewjob?jk=c6760acc724444d7'},
 {'title': 'IT분야 경력사원',
  'company': '우리에프아이에스',
  'location': '서울 마포구',
  'link': 'https://kr.indeed.com/viewjob?jk=3e0c51989197632a'},
 {'title': 'IT팀',
  'company': '월드비전',
  'location': '서울 여의도',
  'link': 'https://kr.indeed.com/viewjob?jk=9c2596d303eb394a'},
 {'title': '[메가존] MSA 아키텍처 대규모 .',
  'company': '메가존',
  'location': '서울 강남구',
  'link': 'https://kr.indeed.com/viewjob?jk=3b11ccb293fe543e'},
 {'title': '[Search & Clova] Clova AI 인재',
  'company': '네이버',
  'location': '대한민국',
  'link': 'https://kr.indeed.com/viewjob?jk=92bd400e3b5306ba'},
 {'title': '백엔드 개발 인턴',
  'company': '리디',
  'location': '서울',
  'link': 'https://kr.indeed.com/viewjob?jk=9e2408dd842f3bfd'},
 {'title': '프론트엔드 개발자',
  'compa